# DQN agent with dietkit data

In [1]:
import numpy as np
import pandas as pd
from dqn_meal_planning_environment import MealPlanningEnv
from dietkit import (load_ingredient, load_menu, load_diet)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

from rl.agents import DQNAgent  # NOTE: requires keras-rl2 not keras-rl
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

2023-04-17 15:46:05.871920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
all_ingredients = load_ingredient(sample_language = 'eng')
meal_dict = load_menu(ingredients = all_ingredients, sample_language = 'eng')
example_mealplans = load_diet(menus = meal_dict, num_loads = 100, sample_language = 'eng', sample_name = 'ML')

possible_meals = np.array(list(meal_dict.keys()))
display(possible_meals)

nutrition_data = pd.DataFrame([meal.nutrition for meal in meal_dict.values()], index=possible_meals)
display(nutrition_data)

array(['S watermelon punch (without milks)', 'Dried persimmon',
       'S kyogo grape (100g)', ..., 'Green onion kimchi',
       'Pickled paprika', 'empty'], dtype='<U72')

,Energy,Protein,Fat,Carbohydrate,Total Dietary,Calcium,Iron,Sodium,Vitamin A,Vitamin B1 (Thiamine),Vitamin B2 (Rivoflavin),Vitamin C,Linoleic Acid,Alpha-Linolenic Acid
S watermelon punch (without milks),55.00,0.7900,0.05000,13.8180,0.200,7.200,0.18600,0.600,71.083333,0.024000,0.03000,0.0000,14.2500,2.9300
Dried persimmon,76.50,0.5820,0.04800,20.5830,3.360,6.300,0.19200,0.900,13.725000,0.003000,0.06720,1.5330,0.0000,8.6010
S kyogo grape (100g),61.00,0.4300,0.04000,16.3800,0.400,4.000,0.17000,1.000,2.166667,0.026000,0.04700,0.1300,11.8900,6.0900
S dried persimmon,85.60,0.7720,0.03200,22.9800,3.400,5.600,0.24800,0.400,9.533333,0.003600,0.09720,1.0680,0.0000,5.0440
S fruit punch,101.05,3.6700,3.36450,14.8900,0.790,117.650,0.19850,36.000,91.166667,0.040350,0.19110,1.8925,86.6605,10.6925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ponytail radish kimchi,10.25,0.5050,0.05000,1.9725,0.775,10.500,0.16750,173.000,5.541667,0.003250,0.01900,0.4125,19.7050,9.4775
Kohlrabi watery kimchi,5.50,0.2900,0.01250,1.2900,0.600,10.500,0.06750,1.750,0.250000,0.203750,0.00875,0.0000,1.9650,5.1100
Green onion kimchi,12.25,0.6250,0.14500,2.1275,1.150,15.000,0.18250,163.750,26.854167,0.012750,0.02850,0.5550,79.8050,17.1425
Pickled paprika,7.94,0.2017,0.03005,1.9859,0.348,1.675,0.08145,67.204,4.850000,0.003675,0.02709,24.2820,12.4483,6.5448


In [4]:
num_meals = 21
env = MealPlanningEnv(possible_meals, nutrition_data, num_meals)

states = env.observation_space['nutrition_history'].shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=(1, *states)))
    model.add(Dense(24, activation='relu'))
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    return model

model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1, 21, 24)         360       
                                                                 
 dense_4 (Dense)             (None, 1, 21, 24)         600       
                                                                 
 flatten_1 (Flatten)         (None, 504)               0         
                                                                 
 dense_5 (Dense)             (None, 3235)              1633675   
                                                                 
Total params: 1,634,635
Trainable params: 1,634,635
Non-trainable params: 0
_________________________________________________________________


In [5]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy, 
        nb_actions=actions, 
        nb_steps_warmup=10, 
        target_model_update=1e-2
    )
    return dqn

dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=1000, visualize=False, verbose=1)

2023-04-17 15:46:32.091324: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_5_1/kernel/Assign' id:337 op device:{requested: '', assigned: ''} def:{{{node dense_5_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5_1/kernel, dense_5_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 1000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 14:13 - reward: -22568.9261

/usr/local/anaconda3/envs/rlclass/lib/python3.9/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-04-17 15:46:32.488264: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_5/BiasAdd' id:233 op device:{requested: '', assigned: ''} def:{{{node dense_5/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_5/MatMul, dense_5/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-17 15:46:32.507460: W tensorflow/c/c_api.cc:300] Operation '{name:'total_1/Assign' id:494 op device:{requested: '', assigned: ''} def:{{{node total_1/Assign}} = AssignVariableOp[_has_manual_c

 1000/10000 [==>...........................] - ETA: 4:00 - reward: -12810.0195done, took 26.771 seconds


In [6]:
results = dqn.test(env, nb_episodes=1, visualize=True)
print(np.mean(results.history['episode_reward']))

Testing for 1 episodes ...
Step: 1
Chosen Meal: Yoghurt fruit salad
Meal History: ['Yoghurt fruit salad' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty']
Nutrition History: [[5.290e+01 1.610e+00 1.030e+00 1.004e+01 9.400e-01 3.955e+01 1.200e-01
  2.315e+01 1.496e+01 1.000e-02 1.100e-01 1.180e+00 4.057e+01 5.430e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00

None

Step: 2
Chosen Meal: Yoghurt fruit salad
Meal History: ['Yoghurt fruit salad' 'Yoghurt fruit salad' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty']
Nutrition History: [[5.290e+01 1.610e+00 1.030e+00 1.004e+01 9.400e-01 3.955e+01 1.200e-01
  2.315e+01 1.496e+01 1.000e-02 1.100e-01 1.180e+00 4.057e+01 5.430e+00]
 [5.290e+01 1.610e+00 1.030e+00 1.004e+01 9.400e-01 3.955e+01 1.200e-01
  2.315e+01 1.496e+01 1.000e-02 1.100e-01 1.180e+00 4.057e+01 5.430e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 

None

Step: 3
Chosen Meal: S mushroom broccoli vegetable porridge
Meal History: ['Yoghurt fruit salad' 'Yoghurt fruit salad'
 'S mushroom broccoli vegetable porridge' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty' 'empty'
 'empty' 'empty' 'empty' 'empty' 'empty']
Nutrition History: [[5.2900e+01 1.6100e+00 1.0300e+00 1.0040e+01 9.4000e-01 3.9550e+01
  1.2000e-01 2.3150e+01 1.4960e+01 1.0000e-02 1.1000e-01 1.1800e+00
  4.0570e+01 5.4300e+00]
 [5.2900e+01 1.6100e+00 1.0300e+00 1.0040e+01 9.4000e-01 3.9550e+01
  1.2000e-01 2.3150e+01 1.4960e+01 1.0000e-02 1.1000e-01 1.1800e+00
  4.0570e+01 5.4300e+00]
 [1.0727e+02 2.3300e+00 1.3600e+00 2.1310e+01 3.8000e-01 5.5800e+00
  5.4000e-01 7.1100e+01 1.0390e+01 7.0000e-02 8.0000e-02 1.5200e+00
  5.1534e+02 9.6700e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.000

...

Step: 20
Chosen Meal: Yoghurt fruit salad
Meal History: ['Yoghurt fruit salad' 'Yoghurt fruit salad'
 'S mushroom broccoli vegetable porridge' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'empty']
Nutrition History: [[5.2900e+01 1.6100e+00 1.0300e+00 1.0040e+01 9.4000e-01 3.9550e+01
  1.2000e-01 2.3150e+01 1.4960e+01 1.0000e-02 1.1000e-01 1.1800e+00
  4.0570e+01 5.4300e+00]
 [5.2900e+01 1.6100e+00 1.0300e+00 1.0040e+01 9.4000e-01 3.9550e+01
  1.2000e-01 2.3150e+01 1.4960e+01 1.0000e-02 1.1000e-01 1.1800e+00
  4.0570e+01 5.4300e+00]
 [1.0727e+02 2.3300e+00 1.3600e+00 2.1310e+01 3.8000e-01 5.5800e+00
  5.4000e-01 7.1100e+01 1.0390e+01 7.0000e-02 8.0000e-02 1.

None

Step: 21
Chosen Meal: Yoghurt fruit salad
Meal History: ['Yoghurt fruit salad' 'Yoghurt fruit salad'
 'S mushroom broccoli vegetable porridge' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad' 'Yoghurt fruit salad'
 'Yoghurt fruit salad' 'Yoghurt fruit salad']
Nutrition History: [[5.2900e+01 1.6100e+00 1.0300e+00 1.0040e+01 9.4000e-01 3.9550e+01
  1.2000e-01 2.3150e+01 1.4960e+01 1.0000e-02 1.1000e-01 1.1800e+00
  4.0570e+01 5.4300e+00]
 [5.2900e+01 1.6100e+00 1.0300e+00 1.0040e+01 9.4000e-01 3.9550e+01
  1.2000e-01 2.3150e+01 1.4960e+01 1.0000e-02 1.1000e-01 1.1800e+00
  4.0570e+01 5.4300e+00]
 [1.0727e+02 2.3300e+00 1.3600e+00 2.1310e+01 3.8000e-01 5.5800e+00
  5.4000e-01 7.1100e+01 1.0390e+01 7.0000e-02

None

Episode 1: reward: -424124.012, steps: 21
-424124.0116452261
